In [1]:
lines=[]
file1=open("StopWords_Auditor.txt")
for line in file1:
    lines.append(line.strip())
file2=open("StopWords_DatesandNumbers.txt")
for line in file2:
    lines.append(line.strip())
file3=open("StopWords_Generic.txt")
for line in file3:
    lines.append(line.strip())
file4=open("StopWords_GenericLong.txt")
for line in file4:
    lines.append(line.strip())
file5=open("StopWords_Geographic.txt")
for line in file5:
    lines.append(line.strip())
file6=open("StopWords_Names.txt")
for line in file6:
    lines.append(line.strip())
file7=open("StopWords_Currencies.txt")
for line in file7:
    lines.append(line.strip())

In [2]:
neg=[]
fileobj=open("negative-words.txt")
for i in fileobj:
    neg.append(i.strip())

In [3]:
positive=[]
fileobj=open("positive-words.txt")
for i in fileobj:
    positive.append(i.strip())

In [4]:
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import pandas as pd
df=pd.read_excel('Input.xlsx')
df.head(5)

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [6]:
def link(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',}
    r = requests.get(url, headers=headers)
    htmldata=r.text
    soup = BeautifulSoup(htmldata, 'html.parser')
    data = ''
    h=[]
    for data in soup.find_all("p"):
        h.append(data.get_text()) 
    return ''.join(h).replace('\xa0','').replace('Introduction','')

In [7]:
df['para']=df['URL'].apply(link)

In [8]:
df.head(3)

,URL_ID,URL,para
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ..."
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...


# 1.1 positive

In [9]:
def positive(alpha):
    positive=[]
    fileobj=open("positive-words.txt")
    for i in fileobj:
        positive.append(i.strip())
    su=[]
    tex=alpha.split(' ')
    tokens_without_sw = [i for i in tex if not i in lines]
    for i in tokens_without_sw:
        if i in positive:
            su.append(1)
    return sum(su)

In [10]:
df['positive score']=df['para'].apply(positive)

# 1.2 negative

In [11]:
def negative(alpha):
    negative=[]
    fileobj=open("negative-words.txt")
    for i in fileobj:
        negative.append(i.strip())
    su=[]
    tex=alpha.split(' ')
    tokens_without_sw = [i for i in tex if not i in lines]
    for i in tokens_without_sw:
        if i in negative:
            su.append(-1)
    return -1*sum(su)

In [12]:
df['negative']=df['para'].apply(negative)

# 1.3 polarity score

In [13]:
df['polarity score']=(df['positive score']-df['negative'])/ ((df['positive score'] + df['negative']) + 0.000001)

In [14]:
df.head(5)

,URL_ID,URL,para,positive score,negative,polarity score
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...,55,26,0.358025
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",45,28,0.232877
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...,57,29,0.325581
3,40.0,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-than...,39,14,0.471698
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention th...,41,18,0.389831


# 1.4 Subjectivity Score

In [19]:
import string
notalpha=[]
for i in string.punctuation:
    notalpha.append(i)
    notalpha.append('–')

In [20]:
def clean(alpha):    
    tex=alpha.split(' ')
    tokens_without_sw = [i for i in tex if not i in lines]
    pf=[x for x in tokens_without_sw if x not in notalpha]
    return len(pf) 

In [21]:
df['Subjectivity Score']=(df['positive score'] + df['negative'])/(df['para'].apply(clean)+0.000001)

In [22]:
df.head(3)

,URL_ID,URL,para,positive score,negative,polarity score,Subjectivity Score
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...,55,26,0.358025,0.079024
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",45,28,0.232877,0.108955
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...,57,29,0.325581,0.095028


# 2.1 Average Sentence Length 

In [23]:
number_of_sentences = sent_tokenize(df['para'][1])
print(len(number_of_sentences))

66


In [24]:
def count_of_sentences(sentences):
    number_of_sentences = sent_tokenize(sentences)
    return len(number_of_sentences)

In [25]:
def count_of_words(sentences):
    number_of_words=sentences.split()
    of=[i for i in number_of_words if i not in notalpha]
    return len(of)

In [26]:
df['Average Sentence Length'] = df['para'].apply(count_of_words)/ df['para'].apply(count_of_sentences)

# 2.2 Percentage of Complex words

In [27]:
def complex_word(word):
    syllable_count=0
    for w in word:
          if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
                syllable_count=syllable_count+1
    return syllable_count

In [28]:
def complexword(alpha):
    zeta=alpha.split()
    lis=list(map(complex_word,zeta))
    v=[]
    for i in lis:
        if i>=2:
            v.append(1)
    return sum(v)

In [29]:
df['para'].apply(complexword)

0      1061
1       691
2       950
3       815
4       870
       ... 
109     480
110     589
111     577
112     304
113     529
Name: para, Length: 114, dtype: int64

In [30]:
df['Percentage of Complex words']=df['para'].apply(complexword)/df['para'].apply(count_of_words)

In [31]:
df.head(3)

,URL_ID,URL,para,positive score,negative,polarity score,Subjectivity Score,Average Sentence Length,Percentage of Complex words
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...,55,26,0.358025,0.079024,34.840000,0.609070
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",45,28,0.232877,0.108955,21.000000,0.498557
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...,57,29,0.325581,0.095028,25.507692,0.572979


# 2.3 Fog Index

In [32]:
df['fog index']=0.4*(df['Average Sentence Length']+df['Percentage of Complex words'])

In [33]:
df.head(3)

,URL_ID,URL,para,positive score,negative,polarity score,Subjectivity Score,Average Sentence Length,Percentage of Complex words,fog index
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...,55,26,0.358025,0.079024,34.840000,0.609070,14.179628
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",45,28,0.232877,0.108955,21.000000,0.498557,8.599423
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...,57,29,0.325581,0.095028,25.507692,0.572979,10.432269


# 3 Average Number of Words Per Sentence

In [34]:
def count_of_words(sentences):
    number_of_words=sentences.split()
    of=[i for i in number_of_words if i not in notalpha]
    return len(of)

In [35]:
def count_of_sentences(sentences):
    number_of_sentences = sent_tokenize(sentences)
    return len(number_of_sentences)

In [36]:
df['Average Number of Words Per Sentence'] = df['para'].apply(count_of_words)/ df['para'].apply(count_of_sentences)

# 4 Complex Word Count

In [37]:
def complex_word(word):
    syllable_count=0
    for w in word:
          if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
                syllable_count=syllable_count+1
    return syllable_count

In [38]:
def complexword(alpha):
    zeta=alpha.split()
    lis=list(map(complex_word,zeta))
    v=[]
    for i in lis:
        if i>=2:
            v.append(1)
    return sum(v)

In [39]:
df['Complex Word Count']=df['para'].apply(complexword)

# 5 Word Count

In [40]:
import nltk

In [41]:
from nltk.corpus import stopwords

In [42]:
stop_words = set(stopwords.words('english'))

In [43]:
from nltk.tokenize import word_tokenize

In [44]:
def clean_proper_count(beta):
    word_tokens = word_tokenize(beta)
    tokens_without_sw = [i for i in word_tokens if not i in stop_words]
    x = [''.join(i for i in s if i not in notalpha) for s in tokens_without_sw]
    y = [s for s in x if s]
    return len(y)

In [45]:
df['Word Count']=df['para'].apply(clean_proper_count)

# 6 Syllable Count Per Word

In [46]:
def complex_word(word):
    if word[-2:]=='ed' or word[-2:]=='es':
        word=word[0:-2]
    syllable_count=0
    for w in word:
          if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
                syllable_count=syllable_count+1
    return syllable_count

In [47]:
def complexword(alpha):
    zeta=alpha.split()
    return sum(list(map(complex_word,zeta)))

In [48]:
df['Syllable Count Per Word']=df['para'].apply(complexword)

In [49]:
df

,URL_ID,URL,para,positive score,negative,polarity score,Subjectivity Score,Average Sentence Length,Percentage of Complex words,fog index,Average Number of Words Per Sentence,Complex Word Count,Word Count,Syllable Count Per Word
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...,55,26,0.358025,0.079024,34.840000,0.609070,14.179628,34.840000,1061,1159,3749
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",45,28,0.232877,0.108955,21.000000,0.498557,8.599423,21.000000,691,803,2546
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...,57,29,0.325581,0.095028,25.507692,0.572979,10.432269,25.507692,950,1020,3425
3,40.0,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-than...,39,14,0.471698,0.073204,20.746667,0.523779,8.508178,20.746667,815,912,2930
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention th...,41,18,0.389831,0.068208,30.777778,0.523466,12.520497,30.777778,870,1011,3162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of an...,18,23,-0.121951,0.084711,22.842105,0.552995,9.358040,22.842105,480,549,1708
110,147.0,https://insights.blackcoffer.com/the-future-of...,An investment is a resource or thing procured ...,29,8,0.567568,0.061056,26.950000,0.546382,10.998553,26.950000,589,670,2119
111,148.0,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision ...,25,35,-0.166667,0.094937,18.295082,0.517025,7.524843,18.295082,577,696,2167
112,149.0,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process ...,23,2,0.840000,0.085911,33.875000,0.560886,13.774354,33.875000,304,327,1144


# 7 Personal Pronouns

In [50]:
def find_pronouns(alpha):
    import re
    le=re.findall(r"\b I \b|\b we \b|\b We \b|\b my \b|\b My \b|\b ours \b|\b Ours \b|\b us \b",alpha)
    return len(le)

In [51]:
df['Personal Pronouns']=df['para'].apply(find_pronouns)

In [52]:
df.head(3)

,URL_ID,URL,para,positive score,negative,polarity score,Subjectivity Score,Average Sentence Length,Percentage of Complex words,fog index,Average Number of Words Per Sentence,Complex Word Count,Word Count,Syllable Count Per Word,Personal Pronouns
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,“If anything kills over 10 million people in t...,55,26,0.358025,0.079024,34.840000,0.609070,14.179628,34.840000,1061,1159,3749,0
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",45,28,0.232877,0.108955,21.000000,0.498557,8.599423,21.000000,691,803,2546,6
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,AI is rapidly evolving in the employment secto...,57,29,0.325581,0.095028,25.507692,0.572979,10.432269,25.507692,950,1020,3425,2


# 8 Average Word Length

In [53]:
def alphabet_count(sigma):
    import string
    d=[]
    for i in sigma:
        d.append(i)
    l=[]
    for i in d:
        if i !=' ':
            l.append(i)
    f=[i for i in l if i not in string.punctuation]
    return len(f)

In [54]:
def count_of_words(sentences):
    number_of_words=sentences.split()
    of=[i for i in number_of_words if i not in notalpha]
    return len(of)

In [55]:
df['Average word Length']=df['para'].apply(alphabet_count)/df['para'].apply(count_of_words)

In [56]:
df_answer=df.drop('para',axis=1)

In [57]:
df_answer.head(5)

,URL_ID,URL,positive score,negative,polarity score,Subjectivity Score,Average Sentence Length,Percentage of Complex words,fog index,Average Number of Words Per Sentence,Complex Word Count,Word Count,Syllable Count Per Word,Personal Pronouns,Average word Length
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,55,26,0.358025,0.079024,34.840000,0.609070,14.179628,34.840000,1061,1159,3749,0,5.765786
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,45,28,0.232877,0.108955,21.000000,0.498557,8.599423,21.000000,691,803,2546,6,4.899711
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,57,29,0.325581,0.095028,25.507692,0.572979,10.432269,25.507692,950,1020,3425,2,5.486731
3,40.0,https://insights.blackcoffer.com/will-machine-...,39,14,0.471698,0.073204,20.746667,0.523779,8.508178,20.746667,815,912,2930,11,4.904242
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,41,18,0.389831,0.068208,30.777778,0.523466,12.520497,30.777778,870,1011,3162,9,5.178700
